In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import shutil
import os

from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from configparser import ConfigParser

RANDOM_STATE=123

FEATURES = ['est_diameter_min', 'est_diameter_max', 'relative_velocity',
            'miss_distance', 'absolute_magnitude']
LABEL = 'hazardous'
COLUMNS =  ['id', 'name'] + FEATURES + [LABEL]

parser = ConfigParser()
parser.read('pass.cfg')
mysql_pass = parser.get('mysql', 'password')

Helpful functions

In [2]:
def df2sqlstring(df):
    '''Concats dataframe rows into a single-string, compatible to SQL-syntax'''
    tuples = list(df.itertuples(index=False, name=None))
    return ','.join(['(' + ','.join([str(w) for w in t]) + ')' for t in tuples])

def mysql2pandas_df(db, table, mysql_user, mysql_pass, columns):
    '''Loads MySQL table to pandas dataframe'''
    db_connection = mysql.connector.connect(user="root", password=mysql_pass)
    db_cursor = db_connection.cursor()
    db_cursor.execute(f'USE {db}')
    db_cursor.execute(f'SELECT {",".join(columns)} FROM {table}')
    df = pd.DataFrame(db_cursor.fetchall())
    df.columns = columns
    return df    
    
def pandas_df2mysql(df, db, table, mysql_user, mysql_pass, col_types):
    '''Loads pandas dataframe to mysql table'''
    columns = ', '.join(df.columns)
    columns_and_types = ', '.join([f'{df.columns[i]} {col_types[i]}' for i in range(len(df.columns))])
    db_connection = mysql.connector.connect(user="root", password=mysql_pass)
    db_cursor = db_connection.cursor()
    db_cursor.execute(f'USE {db}')
    db_cursor.execute(f'DROP TABLE IF EXISTS {table}')
    db_cursor.execute(f'CREATE TABLE {table}({columns_and_types});');
    db_cursor.execute(f'INSERT INTO {table} ({columns}) VALUES ' + df2sqlstring(df) + ';')
    db_cursor.execute('FLUSH TABLES;')

def mysql2pyspark_df(builder, db, table, mysql_user, mysql_pass):
    '''Loads MySQL table to pyspark dataframe'''
    return builder.read.format('jdbc').option('url', f'jdbc:mysql://localhost:3306/{db}') \
    .option('driver', 'com.mysql.cj.jdbc.Driver') \
    .option('dbtable', table) \
    .option('user', mysql_user).option('password', mysql_pass).load()

def pyspark_df2mysql(df, db, table, mysql_user, mysql_pass):
    '''Loads pyspark dataframe to mysql table'''
    return df.write.format('jdbc').option('url', f'jdbc:mysql://localhost:3306/{db}') \
    .option('driver', 'com.mysql.cj.jdbc.Driver') \
    .option('dbtable', table) \
    .option('user', mysql_user).option('password', mysql_pass).save()

#### Split the dataset into train/test to fit the CatBoost and GBTClassifier models in the same conditions
- stratify
- downsample negative examples
- ignore sentry_object feature, because it always equals 0
- ignore orbiting_body feature, because it always equals 'Earth'

In [97]:
# Load the dataset and split to train/test
df = mysql2pandas_df('NeoDB', 'Neo', 'root', mysql_pass, COLUMNS)

df1 = df[df.hazardous == 1]
# Balance positive and negative examples
df2 = df[df.hazardous == 0].sample(frac=1).head(df1.shape[0])
df = pd.concat([df1, df2], axis=0).sample(frac=1)

X_train, X_test, _, _ = train_test_split(df[COLUMNS], df[LABEL], test_size=0.1,
                                         stratify=df[LABEL], random_state=RANDOM_STATE)

n_train, n_train_pos  = X_train.shape[0], sum(X_train[LABEL])
print(f'Train set: {n_train}, positive: {n_train_pos}/{np.round(n_train_pos/n_train*100, 3)}%')
n_test, n_test_pos  = X_test.shape[0], sum(X_test[LABEL])
print(f'Test set: {n_test}, positive: {n_test_pos}/{np.round(n_test_pos/n_test*100, 3)}%')

# Do SMOTE-oversampling of the train set
# sm = SMOTE(random_state=RANDOM_STATE)
# X_train_aug, y_train_aug = sm.fit_resample(X_train[FEATURES], X_train[LABEL])
# X_train_aug[LABEL] = y_train_aug
# print('--')
# n_train_aug, n_train_aug_pos  = X_train_aug.shape[0], sum(X_train_aug[LABEL])
# print(f'Train set with oversampling: \
# {n_train_aug}, positive: {n_train_aug_pos}/{np.round(n_train_aug_pos/n_train_aug * 100, 3)}%')

# Load train/test datasets back to MySQL
column_types = ['INT', 'VARCHAR(1000)'] + ['FLOAT'] * 5 + ['BOOLEAN']
X_train.name = '"' + X_train.name + '"'
pandas_df2mysql(X_train, 'NeoDB', 'NeoTrain', 'root', mysql_pass, column_types)
X_test.name = '"' + X_test.name + '"'
pandas_df2mysql(X_test, 'NeoDB', 'NeoTest', 'root', mysql_pass, column_types)

Train set: 15912, positive: 7956/50.0%
Test set: 1768, positive: 884/50.0%


#### Train CatBoost

In [114]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV, cross_validate
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score
from sklearn.metrics import make_scorer, confusion_matrix

Load train/test data from MySQL

In [99]:
X_train = mysql2pandas_df('NeoDB', 'NeoTrain', 'root', mysql_pass, COLUMNS)
y_train = X_train[LABEL]
X_test = mysql2pandas_df('NeoDB', 'NeoTest', 'root', mysql_pass, COLUMNS)
y_test = X_test[LABEL]

#### Train the CatBoost model

Optimize CatBoost hyperparameters: depth, l2_leaf_reg (5 folds)

The evaluation metric is F-beta (beta=2, recall is more important)

In [110]:
catboost_params = {
    'iterations': [1000],
    'verbose': [False],
    'depth': [2, 3, 4, 5],
    'l2_leaf_reg': [3, 4, 5],
    'random_seed': [RANDOM_STATE]
}

catboost_clf = CatBoostClassifier()
fbeta_scorer = make_scorer(fbeta_score, beta=2)
grid_catboost = GridSearchCV(estimator=catboost_clf, param_grid=catboost_params, cv=5,
                             scoring=fbeta_scorer, n_jobs=-1)
grid_catboost.fit(X_train[FEATURES], y_train)

best_catboost_params = grid_catboost.best_params_
print(f'CatBoost best score: {grid_catboost.best_score_}')
print(f'CatBoost best parameters: {best_catboost_params}')

CatBoost best score: 0.9438765744620958
CatBoost best parameters: {'depth': 2, 'iterations': 1000, 'l2_leaf_reg': 4, 'random_seed': 123, 'verbose': False}


Calculate precision, recall, f1, f-beta to understand results better

In [112]:
scoring = {
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'f-beta': make_scorer(fbeta_score, beta=2)
}

# Calculate precision, recall, f1 and f-beta on cross-validation
catboost_clf = CatBoostClassifier(depth=2, iterations=1000, l2_leaf_reg=4,
                                  verbose=False, random_seed=RANDOM_STATE)
scores = cross_validate(catboost_clf, X_train[FEATURES], y_train,
                        cv=5, n_jobs=-1, scoring=scoring, return_train_score=True)
print('CV-metrics')
for m in scoring.keys():
    print(f'[{m}]')
    train_avg_score = np.round(np.mean(scores['train_' + m][1]), 4)
    test_avg_score = np.round(np.mean(scores['test_' + m][0]), 4)
    print(f'\ttrain avg score: {train_avg_score}')
    print(f'\ttest avg score: {test_avg_score}')

CV-metrics
[precision]
	train avg score: 0.8051
	test avg score: 0.8088
[recall]
	train avg score: 0.9873
	test avg score: 0.9893
[f1]
	train avg score: 0.8869
	test avg score: 0.89
[f-beta]
	train avg score: 0.9445
	test avg score: 0.9471


Calculate prediction for the test dataset

In [115]:
best_catboost_model = CatBoostClassifier(depth=2, iterations=1000, l2_leaf_reg=5, verbose=False,
                                         random_seed=RANDOM_STATE)
best_catboost_model.fit(X_train[FEATURES], y_train)
y_pred_catboost = best_catboost_model.predict(X_test[FEATURES])

tn, fp, fn, tp = confusion_matrix(y_test, y_pred_catboost).ravel()
print(f'Hazardous asteroids: {np.sum(y_test)}')
print(f'Found TP hazardous asteroids: {tp}')
print(f'Found FP hazardous asteroids: {fp}')
print(f'Precision: {np.round(precision_score(y_test, y_pred_catboost), 4)}')
print(f'Recall: {np.round(recall_score(y_test, y_pred_catboost), 4)}')
print(f'f1: {np.round(f1_score(y_test, y_pred_catboost), 4)}')
print(f'f-beta: {np.round(fbeta_score(y_test, y_pred_catboost, beta=2), 4)}')

Hazardous asteroids: 884
Found TP hazardous asteroids: 874
Found FP hazardous asteroids: 197
Precision: 0.8161
Recall: 0.9887
f1: 0.8941
f-beta: 0.9486


Save CatBoost results

In [117]:
# Create and archive parquet files
prediction_df = X_test.copy()
prediction_df['y_pred'] = y_pred_catboost
outdir = './data/catboost_prediction'
if not os.path.exists(outdir):
    os.mkdir(outdir)

prediction_df.to_parquet('data/catboost_prediction/result')
shutil.make_archive('data/catboost_prediction', 'zip', 'data/catboost_prediction')

# Save to MySQL table
column_types_prediction = ['INT', 'VARCHAR(1000)'] + ['FLOAT'] * 5 + ['BOOLEAN'] * 2
prediction_df.name = '"' + prediction_df.name + '"'
pandas_df2mysql(X_train_aug, 'NeoDB', 'NeoCatBoostResult', 'root', mysql_pass, column_types_prediction)

# Save model
best_catboost_model.save_model('models/model_catboost.cbm')

#### Train GBTClassifier [PySpark]

In [45]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.mllib.evaluation import MulticlassMetrics, MulticlassClassificationEvaluator

os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk/'

Load train/test data from MySQL

In [4]:
spark = SparkSession.builder.appName('Neo')\
.config('spark.jars', 'mysql-connector-java-8.0.30/mysql-connector-java-8.0.30.jar').getOrCreate()

df_train = mysql2pyspark_df(spark, 'NeoDB', 'NeoTrain', 'root', mysql_pass)
df_train = df_train.withColumn('hazardous', df_train.hazardous.cast('float'))
df_test = mysql2pyspark_df(spark, 'NeoDB', 'NeoTest', 'root', mysql_pass)
df_test = df_test.withColumn('hazardous', df_test.hazardous.cast('float'))

X_train_vec = VectorAssembler(inputCols=FEATURES, outputCol='features').transform(df_train)
X_test_vec = VectorAssembler(inputCols=FEATURES, outputCol='features').transform(df_test)

22/08/16 17:30:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Train the GBTClassifier with a cross-validation of the next params:
- maxBins: [8, 16, 32, 64]
- maxDepth: [3, 5, 7]

The evaluation metric is weightedFMeasure with beta=2 (recall is more important)

In [47]:
gbt_clf = GBTClassifier(labelCol='hazardous', featuresCol='features')
evaluator = MulticlassClassificationEvaluator(metricName='weightedFMeasure', beta=2.0, labelCol='hazardous')
grid = ParamGridBuilder() \
     .addGrid(gbt_clf.minInstancesPerNode, [1, 2, 3]) \
     .addGrid(gbt_clf.maxDepth, [5, 7, 9, 11]) \
     .addGrid(gbt_clf.maxBins, [32, 64, 128]) \
     .build()

crossval = CrossValidator(estimator=gbt_clf, estimatorParamMaps=grid, evaluator=evaluator, numFolds=5)
cv_gbt_model = crossval.fit(X_train_vec)

22/08/16 18:46:20 WARN DAGScheduler: Broadcasting large task binary with size 1005.8 KiB
22/08/16 18:46:20 WARN DAGScheduler: Broadcasting large task binary with size 1020.2 KiB
22/08/16 18:46:20 WARN DAGScheduler: Broadcasting large task binary with size 1045.2 KiB
22/08/16 18:46:20 WARN DAGScheduler: Broadcasting large task binary with size 1044.9 KiB
22/08/16 18:46:20 WARN DAGScheduler: Broadcasting large task binary with size 1045.3 KiB
22/08/16 18:46:21 WARN DAGScheduler: Broadcasting large task binary with size 1046.0 KiB
22/08/16 18:46:21 WARN DAGScheduler: Broadcasting large task binary with size 1047.0 KiB
22/08/16 18:46:21 WARN DAGScheduler: Broadcasting large task binary with size 1049.3 KiB
22/08/16 18:46:21 WARN DAGScheduler: Broadcasting large task binary with size 1053.3 KiB
22/08/16 18:46:21 WARN DAGScheduler: Broadcasting large task binary with size 1061.1 KiB
22/08/16 18:46:21 WARN DAGScheduler: Broadcasting large task binary with size 1075.1 KiB
22/08/16 18:46:21 WAR

22/08/16 18:46:44 WARN DAGScheduler: Broadcasting large task binary with size 1150.3 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1150.8 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1151.5 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1152.5 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1154.8 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1158.5 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1164.9 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1177.2 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1198.1 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1234.0 KiB
22/08/16 18:46:45 WARN DAGScheduler: Broadcasting large task binary with size 1292.5 KiB
22/08/16 18:46:45 WAR

22/08/16 18:46:50 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/08/16 18:46:50 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/08/16 18:46:50 WARN DAGScheduler: Broadcasting large task binary with size 2.1 MiB
22/08/16 18:46:50 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:50 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:50 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
22/08/16 18:46:51 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:51 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:51 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:51 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:51 WARN DAGScheduler: Broadcasting large task binary with size 2.2 MiB
22/08/16 18:46:51 WARN DAGScheduler: Broadcasting larg

22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1806.8 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1828.1 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1863.0 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1917.8 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1904.8 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1905.2 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1905.9 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1906.9 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1909.2 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1913.7 KiB
22/08/16 18:47:01 WARN DAGScheduler: Broadcasting large task binary with size 1920.9 KiB
22/08/16 18:47:01 WAR

22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1334.8 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1322.2 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1322.7 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1323.4 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1324.4 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1326.7 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1331.2 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1339.4 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1354.0 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1377.8 KiB
22/08/16 18:47:12 WARN DAGScheduler: Broadcasting large task binary with size 1415.0 KiB
22/08/16 18:47:12 WAR

...

22/08/16 19:10:04 WARN DAGScheduler: Broadcasting large task binary with size 1018.5 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1002.0 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1006.3 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1014.1 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1027.8 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1050.4 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1083.6 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1131.6 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1109.3 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1109.7 KiB
22/08/16 19:10:05 WARN DAGScheduler: Broadcasting large task binary with size 1110.4 KiB
22/08/16 19:10:05 WAR

22/08/16 19:10:10 WARN DAGScheduler: Broadcasting large task binary with size 1948.6 KiB
22/08/16 19:10:10 WARN DAGScheduler: Broadcasting large task binary with size 1919.0 KiB
22/08/16 19:10:10 WARN DAGScheduler: Broadcasting large task binary with size 1919.4 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1920.1 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1921.1 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1923.4 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1927.7 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1935.4 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1948.3 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 1969.7 KiB
22/08/16 19:10:11 WARN DAGScheduler: Broadcasting large task binary with size 2001.6 KiB
22/08/16 19:10:11 WAR

22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1782.9 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1783.6 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1784.6 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1786.9 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1791.2 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1798.9 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1811.0 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1830.8 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1860.7 KiB
22/08/16 19:10:22 WARN DAGScheduler: Broadcasting large task binary with size 1903.0 KiB
22/08/16 19:10:23 WARN DAGScheduler: Broadcasting large task binary with size 1883.8 KiB
22/08/16 19:10:23 WAR

22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1587.5 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1566.8 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1567.3 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1568.0 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1569.0 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1571.0 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1574.7 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1581.8 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1594.4 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1615.7 KiB
22/08/16 19:10:36 WARN DAGScheduler: Broadcasting large task binary with size 1649.3 KiB
22/08/16 19:10:36 WAR

22/08/16 19:10:43 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/08/16 19:10:44 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/08/16 19:10:44 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
22/08/16 19:10:44 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
22/08/16 19:10:44 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB


In [49]:
model_gbt = cv_gbt_model.bestModel

print(f'GBT best parameters')
print(f'minInstancesPerNode: {model_gbt.getMinInstancesPerNode()}')
print(f'maxDepth: {model_gbt.getMaxDepth()}')
print(f'maxBins: {model_gbt.getMaxBins()}')

print(f'GBT best f-beta score: {grid_catboost.best_score_}')

print(f'Max F-beta score: {np.avg(cv_gbt_model.avgMetrics)}') # or np.max?



Max F-beta score: 0.8968063653627842
Best model parameters
minInstancesPerNode: 2
maxDepth: 11
maxBins: 128


In [50]:
y_pred_gbt = model_gbt.transform(X_test_vec)
y_pred_gbt.select('name', 'prediction', 'hazardous').show(5)

22/08/16 19:11:26 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
+----------+----------+---------+
|      name|prediction|hazardous|
+----------+----------+---------+
|(2021 CR1)|       0.0|      0.0|
|(2019 XN2)|       0.0|      0.0|
| (2016 JE)|       1.0|      0.0|
|(2018 MG7)|       1.0|      1.0|
|(2019 GV5)|       0.0|      0.0|
+----------+----------+---------+
only showing top 5 rows



In [51]:
def print_all_metrics_spark(df_with_pred):
    dd = df_with_pred.rdd
    scoreAndLabels = dd.map(lambda t: (t.prediction, t.hazardous))
    metrics = MulticlassMetrics(scoreAndLabels)
    y_true = df_with_pred.agg({'hazardous': 'sum'}).collect()[0][0]
    y_pred = df_with_pred.agg({'prediction': 'sum'}).collect()[0][0]
    print(f'True positive values: {y_true}, found positive values: {y_pred}')
    print(f'Precision: {np.round(metrics.precision(1.0), 5)}')
    print(f'Recall: {np.round(metrics.recall(1.0), 5)}')
    print(f'F1: {np.round(metrics.fMeasure(1.0), 5)}')

print_all_metrics_spark(y_pred_gbt)

22/08/16 19:11:28 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
22/08/16 19:11:29 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
True positive values: 884.0, found positive values: 2244.0
22/08/16 19:11:29 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
Precision: 0.33333
Recall: 0.84615
F1: 0.47826


Save GBTClassifier results

In [248]:
# Create and archive parquet files
out_dir = './data/gbt_prediction'
y_pred_gbt.write.mode('overwrite').parquet(out_dir)
shutil.make_archive(out_dir, 'zip', out_dir)

# Save to MySQL table
pyspark_df2mysql(y_pred_gbt.select(COLUMNS + ['prediction']), 'NeoDB', 'NeoGBTResult', 'root', mysql_pass)

# Save model
model_gbt.save('models/model_gbt')